In [20]:
import numpy as np
import random
from PIL import Image
from tensorflow.keras.models import load_model

In [21]:
bot = load_model('bots/blackjack_bot_5000epochs.h5')

In [22]:
# Game logic functions
def create_deck():
    deck = []
    for suit in ['hearts', 'diamonds', 'clubs', 'spades']:
        for rank in range(2, 11):
            deck.append((str(rank), suit))
        for face in ['jack', 'queen', 'king', 'ace']:
            deck.append((face, suit))
    random.shuffle(deck)
    return deck

def calculate_score(hand):
    value_map = {'2': 2, '3': 3, '4': 5, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'jack': 10, 'queen': 10, 'king': 10, 'ace': 11}
    score = sum(value_map[card[0]] for card in hand)
    aces = sum(1 for card in hand if card[0] == 'ace')
    while score > 21 and aces:
        score -= 10
        aces -= 1
    return score

# New function I added to return whether the player has a "usable" ace, meaning the ace can be wither 1 or 11 without going over 21
# This is necessary information for the bot
def has_usable_ace(hand):
    """Check if the hand contains a usable ace (i.e., can count as 11 without busting)."""
    score = calculate_score(hand)
    # A usable ace is one where the ace is counted as 11 and the total score doesn't exceed 21
    return any(card[0] == 'ace' for card in hand) and score <= 21

# Function to get the image paths for the cards
def get_card_image(card):
    rank, suit = card
    return f"cards/{suit}_{rank}.png"

# Resize card image to be smaller
def resize_image(img_path, size=(400, 600)):  # Smaller size
    img = Image.open(img_path)
    img = img.resize(size)
    return img

# Initialize global state
deck = []
player_hand = []
dealer_hand = []
def reset_game():
    global deck, player_hand, dealer_hand
    deck = create_deck()
    # Give two cards to both the player and the dealer at the start
    player_hand = [deck.pop(), deck.pop()]
    dealer_hand = [deck.pop(), deck.pop()]
    player_images = [resize_image(get_card_image(card)) for card in player_hand]
    dealer_images = [resize_image(get_card_image(dealer_hand[0]))]
    result = "Game in progress..."
    player_score = calculate_score(player_hand)
    dealer_score = calculate_score(dealer_hand[:1])
    recommendation = "Hit"
    return player_images, dealer_images, result, player_score, dealer_score, recommendation

In [23]:
# Dealing everyone a new hand
reset_game()
print(player_hand)
print(dealer_hand)

[('9', 'hearts'), ('queen', 'spades')]
[('6', 'diamonds'), ('10', 'diamonds')]


In [24]:
# The bot predicts off a "state" that is just a numpy array of 3 numbers
# A state of [19, 10, 0] means the player's card value total is 19, the dealer's card value total is 10, and the player's hand does NOT include a usable ace
# The dealer's card value total would only include the face-up card(s). There is still a face-down card that the bot/player can't see.
def get_state(player_hand, dealer_hand):
    player_hand_sum = calculate_score(player_hand)
    dealer_hand_sum = calculate_score(dealer_hand[1:]) # Assumes the dealer's first card will be the face-down one, so this excludes from the sum that the bot will see
    player_usable_ace = int(has_usable_ace(player_hand))
    return np.array([player_hand_sum, dealer_hand_sum, player_usable_ace])

get_state(player_hand, dealer_hand)

array([19, 10,  0])

In [26]:
# The bot makes a decision (prediction) based on the state
# The prediction has 2 numbers: the predicted value of choosing 'stand', and the predicted value of choosing 'hit', in that order
# [[ 0.06137741 -0.7087431 ]] would mean it thinks 'stand' is a good idea, and 'hit' is a very bad idea
# np.argmax() extracts whichever one was higher.

def bot_decision(bot, state):
    decision = bot.predict(state[np.newaxis], verbose=False)
    print(decision)
    return "Hit" if np.argmax(decision) == 1 else "Stand"

bot_decision(bot, get_state(player_hand, dealer_hand))

[[ 0.06137741 -0.7087431 ]]


'Stand'

In [ ]:
# The bot still loses more than it wins, but thankfully it's at least trained well enough to avoid really obvious mistakes, like hitting at 19.